## Data Cleaning

In [1]:
import pandas as pd
from datetime import datetime

### Airports Dataset

In [2]:
airports = pd.read_csv('data/airports.csv')
airports.columns = airports.columns.str.lower()
airports.head()

,iata_code,airport,city,state,country,latitude,longitude
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


### Flights Dataset

In [3]:
flights = pd.read_csv('data/flights.csv')

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
flights.columns = flights.columns.str.lower()

# merge flights with airports for origin
flights = flights.merge(right=airports, how='inner', left_on='origin_airport', right_on='iata_code')

flights.rename(columns={'latitude': 'origin_latitude', 'longitude': 'origin_longitude'}, inplace=True)

# merge flights with airports for destination
flights = flights.merge(right=airports, how='inner', left_on='destination_airport', right_on='iata_code',
                        suffixes=('_origin', '_destination'))

flights.rename(columns={'latitude': 'destination_latitude', 'longitude': 'destination_longitude'}, inplace=True)

flights.head()

,year,month,day,day_of_week,airline,flight_number,tail_number,origin_airport,destination_airport,scheduled_departure,...,country_origin,origin_latitude,origin_longitude,iata_code_destination,airport_destination,city_destination,state_destination,country_destination,destination_latitude,destination_longitude
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
1,2015,1,1,4,AS,108,N309AS,ANC,SEA,45,...,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
2,2015,1,1,4,DL,1560,N3743H,ANC,SEA,45,...,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
3,2015,1,1,4,AS,136,N431AS,ANC,SEA,135,...,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931
4,2015,1,1,4,AS,134,N464AS,ANC,SEA,155,...,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931


In [5]:
# remove columns for the stated reasons (primarily irrelvance or collinearity)
flights_clean = flights[flights.columns.difference([
    'year', # all are 2015
    'flight_number', # irrelevant
    'tail_number', # irrelevant
    'departure_time', # collinearity: scheduled_departure + departure_delay
    'wheels_off', # co: departure_time + taxi_out
    'elapsed_time', # co: scheduled_time + departure_delay
    'air_time', # co: elapsed_time - taxi_out - taxi_in
    'wheels_on', # co: arrival_time - taxi_in
    'arrival_time', # co: scheduled_arrival
    # leave in scheduled_arrival because of time change
    'arrival_delay', # co: departure_delay
    'diverted', # occurs after delay, can't be used to predict delay time
    'cancelled', # occurs after delay
    'cancellation_reason',
    'air_system_delay',
    'security_delay',
    'airline_delay',
    'late_aircraft_delay',
    'weather_delay',
    'iata_code_origin', 'iata_code_destination', # same as origin_airport
    'airport_origin', 'airport_destination', # airport already described in origin_airport
    'country_origin', 'country_destination' # all are USA
])]

In [6]:
flights_clean.head()

,airline,city_destination,city_origin,day,day_of_week,departure_delay,destination_airport,destination_latitude,destination_longitude,distance,...,origin_airport,origin_latitude,origin_longitude,scheduled_arrival,scheduled_departure,scheduled_time,state_destination,state_origin,taxi_in,taxi_out
0,AS,Seattle,Anchorage,1,4,-11.0,SEA,47.44898,-122.30931,1448,...,ANC,61.17432,-149.99619,430,5,205.0,WA,AK,4.0,21.0
1,AS,Seattle,Anchorage,1,4,-4.0,SEA,47.44898,-122.30931,1448,...,ANC,61.17432,-149.99619,509,45,204.0,WA,AK,4.0,17.0
2,DL,Seattle,Anchorage,1,4,-14.0,SEA,47.44898,-122.30931,1448,...,ANC,61.17432,-149.99619,515,45,210.0,WA,AK,4.0,25.0
3,AS,Seattle,Anchorage,1,4,NaN,SEA,47.44898,-122.30931,1448,...,ANC,61.17432,-149.99619,600,135,205.0,WA,AK,NaN,NaN
4,AS,Seattle,Anchorage,1,4,-15.0,SEA,47.44898,-122.30931,1448,...,ANC,61.17432,-149.99619,633,155,218.0,WA,AK,11.0,17.0


In [7]:
def time_to_minutes(n):
    # converts from HHMM time (written as number)
    # to number of minutes elapsed since midnight
    return (n // 100) * 60 + (n % 100)

In [8]:
# convert hhmm columns to time since midnight in minutes
flights_clean.scheduled_departure = flights_clean.scheduled_departure.apply(time_to_minutes)
flights_clean.scheduled_arrival = flights_clean.scheduled_arrival.apply(time_to_minutes)

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
flights_clean.head()

,airline,city_destination,city_origin,day,day_of_week,departure_delay,destination_airport,destination_latitude,destination_longitude,distance,...,origin_airport,origin_latitude,origin_longitude,scheduled_arrival,scheduled_departure,scheduled_time,state_destination,state_origin,taxi_in,taxi_out
0,AS,Seattle,Anchorage,1,4,-11.0,SEA,47.44898,-122.30931,1448,...,ANC,61.17432,-149.99619,270,5,205.0,WA,AK,4.0,21.0
1,AS,Seattle,Anchorage,1,4,-4.0,SEA,47.44898,-122.30931,1448,...,ANC,61.17432,-149.99619,309,45,204.0,WA,AK,4.0,17.0
2,DL,Seattle,Anchorage,1,4,-14.0,SEA,47.44898,-122.30931,1448,...,ANC,61.17432,-149.99619,315,45,210.0,WA,AK,4.0,25.0
3,AS,Seattle,Anchorage,1,4,NaN,SEA,47.44898,-122.30931,1448,...,ANC,61.17432,-149.99619,360,95,205.0,WA,AK,NaN,NaN
4,AS,Seattle,Anchorage,1,4,-15.0,SEA,47.44898,-122.30931,1448,...,ANC,61.17432,-149.99619,393,115,218.0,WA,AK,11.0,17.0


In [10]:
# add column for nth day of year (in addition to month/day columns)
def day_counter(row):
    # convert from month and day to nth day of year (i.e. feb 1 -> 32)
    return (datetime(year=2015, month=row.month, day=row.day) - datetime(year=2014, month=12, day=31)).days

In [11]:
flights_clean['day_of_year'] = flights_clean.apply(day_counter, axis=1)

<ipython-input-11-378aa3fec5ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights_clean['day_of_year'] = flights_clean.apply(day_counter, axis=1)


In [12]:
flights_clean.head()

,airline,city_destination,city_origin,day,day_of_week,departure_delay,destination_airport,destination_latitude,destination_longitude,distance,...,origin_latitude,origin_longitude,scheduled_arrival,scheduled_departure,scheduled_time,state_destination,state_origin,taxi_in,taxi_out,day_of_year
0,AS,Seattle,Anchorage,1,4,-11.0,SEA,47.44898,-122.30931,1448,...,61.17432,-149.99619,270,5,205.0,WA,AK,4.0,21.0,1
1,AS,Seattle,Anchorage,1,4,-4.0,SEA,47.44898,-122.30931,1448,...,61.17432,-149.99619,309,45,204.0,WA,AK,4.0,17.0,1
2,DL,Seattle,Anchorage,1,4,-14.0,SEA,47.44898,-122.30931,1448,...,61.17432,-149.99619,315,45,210.0,WA,AK,4.0,25.0,1
3,AS,Seattle,Anchorage,1,4,NaN,SEA,47.44898,-122.30931,1448,...,61.17432,-149.99619,360,95,205.0,WA,AK,NaN,NaN,1
4,AS,Seattle,Anchorage,1,4,-15.0,SEA,47.44898,-122.30931,1448,...,61.17432,-149.99619,393,115,218.0,WA,AK,11.0,17.0,1


In [13]:
flights_clean.loc[0]

airline                         AS
city_destination           Seattle
city_origin              Anchorage
day                              1
day_of_week                      4
departure_delay              -11.0
destination_airport            SEA
destination_latitude      47.44898
destination_longitude   -122.30931
distance                      1448
month                            1
origin_airport                 ANC
origin_latitude           61.17432
origin_longitude        -149.99619
scheduled_arrival              270
scheduled_departure              5
scheduled_time               205.0
state_destination               WA
state_origin                    AK
taxi_in                        4.0
taxi_out                      21.0
day_of_year                      1
Name: 0, dtype: object

In [14]:
flights_clean.dtypes

airline                   object
city_destination          object
city_origin               object
day                        int64
day_of_week                int64
departure_delay          float64
destination_airport       object
destination_latitude     float64
destination_longitude    float64
distance                   int64
month                      int64
origin_airport            object
origin_latitude          float64
origin_longitude         float64
scheduled_arrival          int64
scheduled_departure        int64
scheduled_time           float64
state_destination         object
state_origin              object
taxi_in                  float64
taxi_out                 float64
day_of_year                int64
dtype: object

### Temperatures Dataset

In [15]:
temps = pd.read_csv('data/city_temperature.csv')
temps

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9
...,...,...,...,...,...,...,...,...
2906322,North America,US,Additional Territories,San Juan Puerto Rico,7,27,2013,82.4
2906323,North America,US,Additional Territories,San Juan Puerto Rico,7,28,2013,81.6
2906324,North America,US,Additional Territories,San Juan Puerto Rico,7,29,2013,84.2
2906325,North America,US,Additional Territories,San Juan Puerto Rico,7,30,2013,83.8


In [16]:
temps.columns = temps.columns.str.lower()
temps_clean = temps[temps.country == 'US'][temps.year == 2015]
temps_clean.drop(columns=['region', 'country', 'state', 'year'], inplace=True)
temps_clean

<ipython-input-16-7d51f5c258f8>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temps_clean = temps[temps.country == 'US'][temps.year == 2015]


,city,month,day,avgtemperature
1458295,Birmingham,1,1,40.4
1458296,Birmingham,1,2,45.0
1458297,Birmingham,1,3,56.6
1458298,Birmingham,1,4,56.9
1458299,Birmingham,1,5,39.8
...,...,...,...,...
2897940,Cheyenne,12,27,6.9
2897941,Cheyenne,12,28,11.8
2897942,Cheyenne,12,29,13.2
2897943,Cheyenne,12,30,11.8


In [17]:
cities = temps_clean.city.unique()

In [18]:
present = flights_clean.city_origin.apply(lambda city: city in cities)

In [19]:
present.mean()

0.708105549798853

In [20]:
flights_cities = flights_clean.city_origin.unique()

In [21]:
# rename temps cities to match up with their names in flights
# i.e., Washington Dulles airport is in Chantilly, VA
renames = {
    'Washington DC': 'Chantilly',
    'Tampa St. Petersburg': 'Tampa',
    'Minneapolis St. Paul': 'Minneapolis',
    'New York City': 'New York',
    'Raleigh Durham': 'Raleigh',
    'Akron Canton': 'Akron',
    'Cincinnati': 'Covington',
    'Rhode Island': 'Providence',
    'Dallas Ft Worth': 'Dallas-Fort Worth'
}

In [22]:
temps_clean = temps_clean.replace(renames)
temps_clean

,city,month,day,avgtemperature
1458295,Birmingham,1,1,40.4
1458296,Birmingham,1,2,45.0
1458297,Birmingham,1,3,56.6
1458298,Birmingham,1,4,56.9
1458299,Birmingham,1,5,39.8
...,...,...,...,...
2897940,Cheyenne,12,27,6.9
2897941,Cheyenne,12,28,11.8
2897942,Cheyenne,12,29,13.2
2897943,Cheyenne,12,30,11.8


In [23]:
temps_clean = pd.concat([
    temps_clean,
    temps_clean.loc[temps_clean.city == 'Dallas-Fort Worth'].replace({'Dallas-Fort Worth': 'Dallas'})
], ignore_index=True)

In [24]:
cities = temps_clean.city.unique()

In [25]:
present = flights_clean.city_origin.apply(lambda city: city in cities)

In [26]:
present.mean()

0.8528817453272264

In [27]:
flights_clean.head()

,airline,city_destination,city_origin,day,day_of_week,departure_delay,destination_airport,destination_latitude,destination_longitude,distance,...,origin_latitude,origin_longitude,scheduled_arrival,scheduled_departure,scheduled_time,state_destination,state_origin,taxi_in,taxi_out,day_of_year
0,AS,Seattle,Anchorage,1,4,-11.0,SEA,47.44898,-122.30931,1448,...,61.17432,-149.99619,270,5,205.0,WA,AK,4.0,21.0,1
1,AS,Seattle,Anchorage,1,4,-4.0,SEA,47.44898,-122.30931,1448,...,61.17432,-149.99619,309,45,204.0,WA,AK,4.0,17.0,1
2,DL,Seattle,Anchorage,1,4,-14.0,SEA,47.44898,-122.30931,1448,...,61.17432,-149.99619,315,45,210.0,WA,AK,4.0,25.0,1
3,AS,Seattle,Anchorage,1,4,NaN,SEA,47.44898,-122.30931,1448,...,61.17432,-149.99619,360,95,205.0,WA,AK,NaN,NaN,1
4,AS,Seattle,Anchorage,1,4,-15.0,SEA,47.44898,-122.30931,1448,...,61.17432,-149.99619,393,115,218.0,WA,AK,11.0,17.0,1


In [28]:
temps_clean.head()

,city,month,day,avgtemperature
0,Birmingham,1,1,40.4
1,Birmingham,1,2,45.0
2,Birmingham,1,3,56.6
3,Birmingham,1,4,56.9
4,Birmingham,1,5,39.8


In [29]:
# merge flights with temps now that we've cleaned temps so that most flights match up
flights_clean = flights_clean.merge(right=temps_clean, how='inner', left_on=['city_origin', 'month', 'day'],
                                    right_on=['city', 'month', 'day'])

flights_clean.rename(columns={'avgtemperature': 'origin_temperature'}, inplace=True)

In [30]:
flights_clean.drop(columns=[])

,airline,city_destination,city_origin,day,day_of_week,departure_delay,destination_airport,destination_latitude,destination_longitude,distance,...,scheduled_arrival,scheduled_departure,scheduled_time,state_destination,state_origin,taxi_in,taxi_out,day_of_year,city,origin_temperature
0,AS,Seattle,Anchorage,1,4,-11.0,SEA,47.44898,-122.30931,1448,...,270,5,205.0,WA,AK,4.0,21.0,1,Anchorage,35.2
1,AS,Seattle,Anchorage,1,4,-4.0,SEA,47.44898,-122.30931,1448,...,309,45,204.0,WA,AK,4.0,17.0,1,Anchorage,35.2
2,DL,Seattle,Anchorage,1,4,-14.0,SEA,47.44898,-122.30931,1448,...,315,45,210.0,WA,AK,4.0,25.0,1,Anchorage,35.2
3,AS,Seattle,Anchorage,1,4,NaN,SEA,47.44898,-122.30931,1448,...,360,95,205.0,WA,AK,NaN,NaN,1,Anchorage,35.2
4,AS,Seattle,Anchorage,1,4,-15.0,SEA,47.44898,-122.30931,1448,...,393,115,218.0,WA,AK,11.0,17.0,1,Anchorage,35.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4681698,NK,Ft. Lauderdale,Atlantic City,31,4,-5.0,FLL,26.07258,-80.15275,977,...,1365,1207,158.0,FL,NJ,9.0,9.0,365,Atlantic City,54.0
4681699,NK,Tampa,Atlantic City,31,4,-2.0,TPA,27.97547,-82.53325,913,...,958,801,157.0,FL,NJ,7.0,11.0,365,Atlantic City,54.0
4681700,NK,Ft. Myers,Atlantic City,31,4,2.0,RSW,26.53617,-81.75517,982,...,549,380,169.0,FL,NJ,4.0,11.0,365,Atlantic City,54.0
4681701,NK,Ft. Myers,Atlantic City,31,4,6.0,RSW,26.53617,-81.75517,982,...,1200,1040,160.0,FL,NJ,3.0,7.0,365,Atlantic City,54.0


In [31]:
# same for destination temps
flights_clean = flights_clean.merge(right=temps_clean, how='inner', left_on=['city_destination', 'month', 'day'],
                    right_on=['city', 'month', 'day'])

flights_clean.rename(columns={'avgtemperature': 'destination_temperature'}, inplace=True)

In [32]:
flights_clean

,airline,city_destination,city_origin,day,day_of_week,departure_delay,destination_airport,destination_latitude,destination_longitude,distance,...,scheduled_time,state_destination,state_origin,taxi_in,taxi_out,day_of_year,city_x,origin_temperature,city_y,destination_temperature
0,AS,Seattle,Anchorage,1,4,-11.0,SEA,47.44898,-122.30931,1448,...,205.0,WA,AK,4.0,21.0,1,Anchorage,35.2,Seattle,32.9
1,AS,Seattle,Anchorage,1,4,-4.0,SEA,47.44898,-122.30931,1448,...,204.0,WA,AK,4.0,17.0,1,Anchorage,35.2,Seattle,32.9
2,DL,Seattle,Anchorage,1,4,-14.0,SEA,47.44898,-122.30931,1448,...,210.0,WA,AK,4.0,25.0,1,Anchorage,35.2,Seattle,32.9
3,AS,Seattle,Anchorage,1,4,NaN,SEA,47.44898,-122.30931,1448,...,205.0,WA,AK,NaN,NaN,1,Anchorage,35.2,Seattle,32.9
4,AS,Seattle,Anchorage,1,4,-15.0,SEA,47.44898,-122.30931,1448,...,218.0,WA,AK,11.0,17.0,1,Anchorage,35.2,Seattle,32.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4099037,EV,Roanoke,Atlanta,27,5,77.0,ROA,37.32547,-79.97543,357,...,83.0,VA,GA,3.0,9.0,331,Atlanta,59.7,Roanoke,49.4
4099038,EV,Roanoke,Atlanta,27,5,-6.0,ROA,37.32547,-79.97543,357,...,83.0,VA,GA,5.0,21.0,331,Atlanta,59.7,Roanoke,49.4
4099039,DL,Roanoke,Atlanta,27,5,-1.0,ROA,37.32547,-79.97543,357,...,83.0,VA,GA,6.0,22.0,331,Atlanta,59.7,Roanoke,49.4
4099040,EV,Allentown,Atlanta,19,6,88.0,ABE,40.65236,-75.44040,692,...,113.0,PA,GA,3.0,22.0,353,Atlanta,37.8,Allentown,35.6


In [33]:
flights_clean.loc[0]

airline                           AS
city_destination             Seattle
city_origin                Anchorage
day                                1
day_of_week                        4
departure_delay                -11.0
destination_airport              SEA
destination_latitude        47.44898
destination_longitude     -122.30931
distance                        1448
month                              1
origin_airport                   ANC
origin_latitude             61.17432
origin_longitude          -149.99619
scheduled_arrival                270
scheduled_departure                5
scheduled_time                 205.0
state_destination                 WA
state_origin                      AK
taxi_in                          4.0
taxi_out                        21.0
day_of_year                        1
city_x                     Anchorage
origin_temperature              35.2
city_y                       Seattle
destination_temperature         32.9
Name: 0, dtype: object

In [34]:
flights_clean.drop(columns=['city_destination', 'city_origin', 'city_x', 'city_y'], inplace=True)

In [35]:
flights_clean.loc[0]

airline                           AS
day                                1
day_of_week                        4
departure_delay                -11.0
destination_airport              SEA
destination_latitude        47.44898
destination_longitude     -122.30931
distance                        1448
month                              1
origin_airport                   ANC
origin_latitude             61.17432
origin_longitude          -149.99619
scheduled_arrival                270
scheduled_departure                5
scheduled_time                 205.0
state_destination                 WA
state_origin                      AK
taxi_in                          4.0
taxi_out                        21.0
day_of_year                        1
origin_temperature              35.2
destination_temperature         32.9
Name: 0, dtype: object

In [36]:
flights_clean.isna().sum()

airline                        0
day                            0
day_of_week                    0
departure_delay            63038
destination_airport            0
destination_latitude           0
destination_longitude          0
distance                       0
month                          0
origin_airport                 0
origin_latitude                0
origin_longitude               0
scheduled_arrival              0
scheduled_departure            0
scheduled_time                 3
state_destination              0
state_origin                   0
taxi_in                    67123
taxi_out                   65176
day_of_year                    0
origin_temperature             0
destination_temperature        0
dtype: int64

In [37]:
flights_clean.dropna(inplace=True)

In [38]:
flights_clean.isna().sum()

airline                    0
day                        0
day_of_week                0
departure_delay            0
destination_airport        0
destination_latitude       0
destination_longitude      0
distance                   0
month                      0
origin_airport             0
origin_latitude            0
origin_longitude           0
scheduled_arrival          0
scheduled_departure        0
scheduled_time             0
state_destination          0
state_origin               0
taxi_in                    0
taxi_out                   0
day_of_year                0
origin_temperature         0
destination_temperature    0
dtype: int64

In [39]:
flights_clean.to_csv('data/flights_clean.csv', index=False)

In [40]:
# dummy variable dataset out of categorical columns
flights_clean_numerical = pd.get_dummies(flights_clean)

In [41]:
flights_clean_numerical.to_csv('data/flights_clean_numerical.csv', index=False)

In [42]:
# subset dummy columns with only those significant in the base model
columns_subset = \
    list(flights_clean.columns.difference(['airline', 'destination_airport',
                                     'origin_airport', 'state_destination', 'state_origin']))
columns_subset += [
    'airline_AS', 'airline_NK', 'airline_UA',
    'destination_airport_BTV', 'destination_airport_DTW', 'destination_airport_FNT', 
    'origin_airport_CMH', 'origin_airport_IAD', 'origin_airport_LNK', 'origin_airport_RIC',
    'state_destination_MI', 'state_destination_VT', 'state_origin_NE'
]
flights_clean_numerical_significant = flights_clean_numerical[columns_subset]

In [43]:
flights_clean_numerical_significant.to_csv('data/flights_clean_numerical_significant.csv', index=False)